<a href="https://colab.research.google.com/github/jmbenedetto/piloting/blob/main/measurements_follow_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GOAL

# PACKAGES

In [176]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import altair as alt
from datetime import date

!pip install sweetviz
import sweetviz as sv

# PARAMETERS

In [177]:
period='2022Q1'

# SCRIPT

## Get measurements data

In [178]:
# connect to google account 
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# get gsheet
sh = gc.open("2022_OKRs and 4DX")
wsh_measurements = sh.worksheet("Measurements follow up")

# get data into dfs
wsh_definitions = sh.worksheet("Measures definition").get_all_values()
df_definitions = pd.DataFrame.from_records(wsh_definitions[1:],columns=wsh_definitions[0])
wsh_measurements = sh.worksheet("Measurements follow up").get_all_values()
df_measurements = pd.DataFrame.from_records(wsh_measurements[1:],columns=wsh_measurements[0])
df_measurements['Week']=pd.to_datetime(df_measurements['Week'],dayfirst=True).astype('str')
# transform Target and Value into Function and Measurement columns
df_measurements=df_measurements.melt(
    id_vars=['MeasureID','MeasurementID','Week'],
    var_name='Function',
    value_name='Measurement')
df = pd.merge(df_definitions,df_measurements,how='left',on='MeasureID')
df['Measure key']="(" + df['Period'] + ") " + df['Measure']
df['Measurement']=df['Measurement'].replace(",",".",regex=True)
df.head(5)

,MeasureID,Measure,Type,UoM,Period,MeasurementID,Week,Function,Measurement,Measure key
0,05a85df9,Weight,Lag,Kg,2022Q1,b4cd08ad,2022-01-10,Target,96.5,(2022Q1) Weight
1,05a85df9,Weight,Lag,Kg,2022Q1,1,2022-01-17,Target,96,(2022Q1) Weight
2,05a85df9,Weight,Lag,Kg,2022Q1,2,2022-01-24,Target,95.5,(2022Q1) Weight
3,05a85df9,Weight,Lag,Kg,2022Q1,3,2022-01-31,Target,95,(2022Q1) Weight
4,05a85df9,Weight,Lag,Kg,2022Q1,4,2022-02-07,Target,94.5,(2022Q1) Weight


In [ ]:
#analyzing the dataset
advert_report = sv.analyze(df)
#display the report
advert_report.show_html('Advertising.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Advertising.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Generate plots

In [217]:
graphs=None
for measure in df['Measure'].unique():
  source = df[(df['Period']==period)&(df['Measure']==measure)]
  if graphs:
    graphs = graphs & alt.Chart(source).mark_bar().encode(
      x=alt.X('Function:N',title=None,axis=alt.Axis(labels=False)),
      y=alt.Y('Measurement:Q',stack=None,title=measure),
      column='Week:O',
      color='Function:N'
    ).properties(
      width=40,
      height=75)
  else:
    graphs = alt.Chart(source).mark_bar().encode(
        x=alt.X('Function:N',title=None,axis=alt.Axis(labels=False)),
        y=alt.Y('Measurement:Q',stack=None,title=measure),
        column='Week:O',
        color='Function:N'
      ).properties(
        width=40,
        height=75)
      
graphs

alt.VConcatChart(...)

In [219]:
graphs=None
for measure in df['Measure'].unique():
  source = df[(df['Period']==period)&(df['Measure']==measure)]
  bar = alt.Chart(source[source['Function']=='Value']).mark_bar().encode(
    x=alt.X('Week:O',title=None),
    y=alt.Y('Measurement:Q',title=measure)
  ).properties(
        width=400,
        height=200)

  tick = (alt.Chart(source[source['Function']=='Target']).
          mark_tick(color='red',
    thickness=2).encode(
    x=alt.X('Week:O',title=None),
    y=alt.Y('Measurement:Q',stack=None,title=measure)
  ).properties(
        width=400,
        height=200))
  
  graph = bar + tick

  if graphs:
    graphs = graphs & graph
  else:
    graphs = graph
      
graphs

alt.VConcatChart(...)